<a href="https://colab.research.google.com/github/SeWonKwon/NLP/blob/main/L04_Document_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문서 분류(Document Classification)

## 데이터 준비

* 문서 분류에 필요한 데이터는 `scikit-learn`이 제공하는 20개의 주제를 가지는 뉴스그룹 데이터를 사용
* 텍스트는 `CounterVectorizer`를 거쳐 DTM(Document Term Matrix)로 변환
* DTM 행렬은 문서에 등장하는 단어들을 빈도 수 별로 표현한 행렬


In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

news = fetch_20newsgroups()

x = news.data
y = news.target

cv = CountVectorizer()
x = cv.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(7919, 130107) (7919,) (3395, 130107) (3395,)


In [45]:
list(cv.vocabulary_.keys())[:10]

['from',
 'lerxst',
 'wam',
 'umd',
 'edu',
 'where',
 'my',
 'thing',
 'subject',
 'what']

In [21]:
print(x_train[0])

  (0, 56979)	1
  (0, 118280)	2
  (0, 50527)	2
  (0, 114688)	1
  (0, 111322)	1
  (0, 68532)	4
  (0, 114731)	1
  (0, 89362)	5
  (0, 76032)	1
  (0, 89860)	1
  (0, 114455)	11
  (0, 68766)	2
  (0, 115475)	6
  (0, 32311)	3
  (0, 66608)	3
  (0, 27436)	3
  (0, 128402)	2
  (0, 35983)	1
  (0, 56283)	1
  (0, 28146)	2
  (0, 123759)	2
  (0, 29620)	2
  (0, 114440)	2
  (0, 28012)	1
  (0, 35805)	2
  :	:
  (0, 76860)	1
  (0, 68740)	2
  (0, 40183)	1
  (0, 121661)	1
  (0, 93664)	2
  (0, 114539)	1
  (0, 79253)	3
  (0, 39084)	1
  (0, 124463)	2
  (0, 65703)	1
  (0, 39707)	1
  (0, 100781)	1
  (0, 87174)	2
  (0, 91194)	1
  (0, 55785)	1
  (0, 121510)	1
  (0, 76843)	2
  (0, 111052)	1
  (0, 108630)	1
  (0, 115883)	1
  (0, 79655)	1
  (0, 26198)	1
  (0, 71186)	1
  (0, 52537)	1
  (0, 29315)	1


## scikit-learn을 이용한 문서 분류

In [22]:
from sklearn.metrics import accuracy_score

### Logistic Regression

<img src='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/capture_images/81ae04166e344109a330421125657a89/c55a9dad-26a4-40dd-9cc4-96262a77f507.png' />

* Logistic Regression은 특성상 다중 분류에는 적합하지 않음

In [23]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(x_train, y_train)
pred = LR.predict(x_test)

acc = accuracy_score(pred, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [24]:
acc

0.8645066273932254

다중 분류이기 때문에 여기에는 어울리지는 않음

### 서포트 벡터 머신 (Support Vector Machine)

* 회귀, 분류, 이상치 탐지 등에 사용되는 지도학습 방법
* 클래스 사이의 경계에 위치한 데이터 포인트를 서포트 벡터(support vector)라고 함
* 각 서포트 벡터가 클래스 사이의 결정 경계를 구분하는데 얼마나 중요한지를 학습
* 각 서포트 벡터 사이의 마진이 가장 큰 방향으로 학습
* 서포트 벡터 까지의 거리와 서포트 벡터의 중요도를 기반으로 예측을 수행


In [26]:
from sklearn.svm import SVC
svm = SVC(kernel='linear')

svm.fit(x_train, y_train)
pred = svm.predict(x_test)

acc = accuracy_score(pred, y_test)
print(acc)

0.8229749631811487


### Naive Bayes

* 베이즈 정리를 적용한 확률적 분류 알고리즘
* 모든 특성들이 독립 임을 가정 (naive 가정)
* 입력 특성에 따라 3개의 분류기 존재
  * 가우시안 나이브 베이즈 분류기
  * 베르누이 나이브 베이즈 분류기
  * 다항 나이브 베이즈 분류기

In [27]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

nb.fit(x_train, y_train)
pred = nb.predict(x_test)
acc = accuracy_score(pred, y_test)
print(acc)

0.81620029455081


#### tf-idf를 이용한 정확도 향상

In [30]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
x_train_tf = tfidf.fit_transform(x_train)
x_test_tf = tfidf.fit_transform(x_test)

nb.fit(x_train_tf, y_train)
pred = nb.predict(x_test_tf)
acc = accuracy_score(pred, y_test)
print(acc)


0.8265095729013254


### 결정 트리(Decision Tree)

* 분류와 회귀에 사용되는 지도 학습 방법
* 데이터 특성으로 부터 추론된 결정 규칙을 통해 값을 예측
* if-then-else 결정 규칙을 통해 데이터 학습
* 트리의 깊이가 깊을 수록 복잡한 모델


In [32]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)
pred = dt.predict(x_test)
acc = accuracy_score(pred, y_test)
print(acc)

0.6486008836524301


### XGBoost

* 트리 기반의 앙상블 기법
* 분류에 있어서 다른 알고리즘보다 좋은 예측 성능을 보여줌
* XGBoost는 GBM 기반이지만, GBM의 단점인 느린 수행 시간과 과적합 규제 부재 등의 문제를 해결
* 병렬 CPU 환경에서 빠르게 학습 가능

In [34]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=30, learning_rate=0.05, max_depth=3)
xgb.fit(x_train, y_train)
pred = xgb.predict(x_test)
acc = accuracy_score(pred, y_test)
print(acc)

0.7272459499263623


##교차 검증

* 일반 검증은 학습 데이터가 테스트 데이터로 사용되지 않음
* 교차 검증은 데이터를 n개의 집합으로 나누어 정확도를 계산해 학습 데이터로 사용된 데이터도 테스트 데이터로 사용
* 교차 검증을 사용하면 일반 검증보다 모델의 일반화가 잘 되어 있는지 평가 가능
* 앞서 구성한 나이브 베이즈 모델을 교차 검증

In [35]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(nb, x, y, cv=5 )
print(scores, scores.mean())

[0.83870968 0.83826779 0.82368537 0.83031374 0.83642794] 0.833480903927519


* 교차 검증을 통해 일반 검증보다 좀 더 일반화된 모델 생성 가능
* 교차 검증은 일반 검증에 비해 n번 검증을 해 비용이 더 많이 소요

## 정밀도와 재현률 

* 정밀도(precision)는 양성 클래스(정답)으로 예측한 샘플이 양성 클래스일 확률을 의미
* 모델이 얼마나 양성 클래스를 잘 예측하는지를 나타냄
* 재현률(recall)은 양성 클래스인 샘플에서 모델이 양성 클래스로 예측한 샘플 비율을 의미하며, 모델이 얼마나 실제 상황을 재현하는지를 나타냄
* 정밀도와 재현율의 가중조화평균인 F1-score라는 지표는 정확도에 비해 더 효과적인 모델 분석 지표로 알려져 있음
* 직접 계산할 수도 있으나, scikit-learn은 이를 편리하게 계산해주는 함수를 제공

* 다중 클래스 분류 문제에서 정밀도와 재현률을 계산할 때는 클래스간의 지표를 어떻게 합칠지 지정 필요

  * None - 클래스간 지표를 합치지 말고 그대로 출력
  * micro - 정밀도와 재현률이 같음, 이로 인해 f1-score도 정밀도, 재현률과 동일
  * macro - 클래스간 지표를 단순 평균한 값
  * weighted - 클래스간 지표를 가중 평균한 값

In [36]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(pred, y_test, average='micro')
recall = recall_score(pred, y_test, average='micro')
f1 = f1_score(pred, y_test, average='micro')

print(precision, recall , f1)

0.7272459499263623 0.7272459499263623 0.7272459499263623


In [37]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(pred, y_test, average='macro')
recall = recall_score(pred, y_test, average='macro')
f1 = f1_score(pred, y_test, average='macro')

print(precision, recall , f1)

0.7245739794860545 0.7606625246238752 0.7368971171958822


## 그리드 검색을 이용한 파라미터 최적화

* 그리드 검색을 사용하면 분류기에 사용하는 파라미터 최적화 가능
* 그리드 검색을 통해 앞서 구성한 나이브 베이즈 모델의 'alpha' 파라미터를 최적화시키는 예제

* `estimator`: 사용 모델 객체     
* `param_grid`: 사용 객체:지정 파라미터 리스트로 구성된 딕셔너리    
* `scoring`: 최적화하고자 하는 성능 지표   
* `cv`: 교차 검증 분할 개수      

In [42]:
from sklearn.model_selection import GridSearchCV

GS = GridSearchCV(estimator=nb, param_grid={'alpha': [0.001, 0.0012, 0.0013, 0.0011]},
                  scoring='accuracy', cv=10)
GS.fit(x, y)

print(GS.best_score_)
print(GS.best_params_)

0.8902240270188363
{'alpha': 0.0013}
